In [0]:
# Databricks notebook source
# MAGIC %sql
# MAGIC select * from main.default.workspace_cost_details_mv;

# COMMAND ----------

# MAGIC %sql
# MAGIC SELECT 
# MAGIC   workspace, 
# MAGIC   SUM(today) AS Todays_cost, 
# MAGIC   SUM(mtd) AS month_till_date_cose, 
# MAGIC   1000 AS expectd_cast
# MAGIC FROM main.default.workspace_cost_details_mv
# MAGIC GROUP BY workspace
# MAGIC order by Todays_cost desc

# COMMAND ----------

import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os
from pyspark.sql.functions import col, sum , current_timestamp
from pyspark.sql.types import DateType

# COMMAND ----------

#code for just email

def send_email(to_email, subject, body):
    smtp_server = 'smtp.office365.com'
    smtp_port = 587
    sender_email = 'syed.sardar@exponentia.ai'
    password = 'zcxsjpyttdvjsphl'

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = ", ".join(to_email)  # Join list of emails with commas
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'html'))

    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, password)
        server.send_message(msg)


# COMMAND ----------

# # ----------------------------------------
# # 1️⃣ Run Query to Get Cost Data
# # ----------------------------------------
# query = """
# SELECT 
#   workspace, 
#   SUM(today) AS Todays_cost, 
#   SUM(mtd) AS month_till_date_cose, 
#   50 AS expectd_cast
# FROM main.default.workspace_cost_details_mv
# GROUP BY workspace
# """
# df = spark.sql(query)
# results = df.collect()

# # ----------------------------------------
# # 2️⃣ Filter rows where Todays_cost > expectd_cast
# # ----------------------------------------
# rows_to_include = [row for row in results if row['Todays_cost'] > row['expectd_cast']]

# # Only proceed if there is any alert-worthy data
# if rows_to_include:

#     # ----------------------------------------
#     # 3️⃣ Generate HTML Table Rows
#     # ----------------------------------------
#     rows_html = ""
#     for row in rows_to_include:
#         rows_html += f"""
#             <tr>
#                 <td style="border: 1px solid #aaa; text-align: left; padding: 4px;">{row['workspace']}</td>
#                 <td style="border: 1px solid #aaa; text-align: right; padding: 4px;">{row['Todays_cost']:.2f}</td>
#                 <td style="border: 1px solid #aaa; text-align: right; padding: 4px;">{row['expectd_cast']:.2f}</td>
#                 <td style="border: 1px solid #aaa; text-align: right; padding: 4px;">{row['month_till_date_cose']:.2f}</td>
#             </tr>
#         """

#     # ----------------------------------------
#     # 4️⃣ Build Final HTML Body
#     # ----------------------------------------
#     dashboard_link = "https://adb-3859797657626704.4.azuredatabricks.net/dashboardsv3/01f04b37da0f11c29b7becaa4cee4759/published?o=3859797657626704&f_page1%7Es1ParamTimeKey=Week&f_page1%7Es1ParamGroupKey=Workspace"  # Replace with real one

#     email_body = f"""
#     <html>
#       <body style="font-family: Arial, sans-serif; font-size: 15px; color: #333;">
#         <p>Dear Team,</p>

#         <p>
#           Below is the overview of cost usage <strong>workspace-wise</strong>, where the actual spend is higher than expected.
#         </p>

#         <h3 style="color: #b30000;">Databricks Cost Alert</h3>

#         <table border="1" cellpadding="6" cellspacing="0" style="border-collapse: collapse; width: 95%; font-size: 14px;">
#           <thead>
#             <tr style="background-color: #f2f2f2;">
#               <th style="border: 1px solid #aaa; text-align: left;">Workspace</th>
#               <th style="border: 1px solid #aaa; text-align: right;">Today's Cost (₹)</th>
#               <th style="border: 1px solid #aaa; text-align: right;">Expected Cost (₹)</th>
#               <th style="border: 1px solid #aaa; text-align: right;">Month Till Date Cost (₹)</th>
#             </tr>
#           </thead>
#           <tbody>
#             {rows_html}
#           </tbody>
#         </table>

#         <p style="margin-top: 20px;">
#           For more details, <a href="{dashboard_link}" style="color: #004080; font-weight: bold;">click here</a>.
#         </p>

#         <p style="margin-top: 30px;">
#           Regards,<br>
#           <strong>Team Exponentia</strong>
#         </p>

#         <p style="color: #888; font-size: 13px; border-top: 1px solid #ccc; padding-top: 10px;">
#           This is an auto-generated email. Please do not reply.
#         </p>
#       </body>
#     </html>
#     """

#     # ----------------------------------------
#     # 5️⃣ Send the email
#     # ----------------------------------------

#     send_email(
#         to_email=['shripati.more@exponentia.ai','syed.sardar@exponentia.ai'],
#         subject='Databricks Cost Alert',
#         body=email_body
#     )

# else:
#     print("✅ No cost alert: Today's cost is within expected range. No email sent.")


# COMMAND ----------


# # ----------------------------------------
# # 1️⃣ Run Query to Get Cost Data
# # ----------------------------------------
# query = """
# SELECT 
#   workspace, 
#   SUM(today) AS Todays_cost, 
#   SUM(mtd) AS month_till_date_cose, 
#   10 AS expectd_cast
# FROM main.default.workspace_cost_details_mv
# GROUP BY workspace
# """
# df = spark.sql(query)
# results = df.collect()

# # ----------------------------------------
# # 2️⃣ Filter rows where Today's cost > expected
# # ----------------------------------------
# rows_to_include = [row for row in results if row['Todays_cost'] > row['expectd_cast']]

# # ----------------------------------------
# # 3️⃣ Build Email Body Based on Condition
# # ----------------------------------------

# dashboard_link = "https://adb-3859797657626704.4.azuredatabricks.net/dashboardsv3/01f04b37da0f11c29b7becaa4cee4759/published?o=3859797657626704&f_page1%7Es1ParamTimeKey=Week&f_page1%7Es1ParamGroupKey=Workspace"  # Replace with real link

# if rows_to_include:
#     # Table Email Body (alert)
#     rows_html = ""
#     for row in rows_to_include:
#         rows_html += f"""
#             <tr>
#                 <td style="border: 1px solid #aaa; text-align: left; padding: 4px;">{row['workspace']}</td>
#                 <td style="border: 1px solid #aaa; text-align: right; padding: 4px;">{row['Todays_cost']:.2f}</td>
#                 <td style="border: 1px solid #aaa; text-align: right; padding: 4px;">{row['expectd_cast']:.2f}</td>
#                 <td style="border: 1px solid #aaa; text-align: right; padding: 4px;">{row['month_till_date_cose']:.2f}</td>
#             </tr>
#         """

#     email_body = f"""
#     <html>
#       <body style="font-family: Arial, sans-serif; font-size: 15px; color: #333;">
#         <p>Dear Team,</p>

#         <p>
#           Below is the overview of cost usage <strong>workspace-wise</strong>, where the actual spend is higher than expected.
#         </p>

#         <h3 style="color: #b30000;">Databricks Cost Alert</h3>

#         <table border="1" cellpadding="6" cellspacing="0" style="border-collapse: collapse; width: 95%; font-size: 14px;">
#           <thead>
#             <tr style="background-color: #f2f2f2;">
#               <th style="border: 1px solid #aaa; text-align: left;">Workspace</th>
#               <th style="border: 1px solid #aaa; text-align: right;">Today's Cost (₹)</th>
#               <th style="border: 1px solid #aaa; text-align: right;">Expected Cost (₹)</th>
#               <th style="border: 1px solid #aaa; text-align: right;">Month Till Date Cost (₹)</th>
#             </tr>
#           </thead>
#           <tbody>
#             {rows_html}
#           </tbody>
#         </table>

#         <p style="margin-top: 20px;">
#           For more details, <a href="{dashboard_link}" style="color: #004080; font-weight: bold;">click here</a>.
#         </p>

#         <p style="margin-top: 30px;">
#           Regards,<br>
#           <strong>Syed Sardar</strong>
#         </p>

#         <p style="color: #888; font-size: 13px; border-top: 1px solid #ccc; padding-top: 10px;">
#           This is an auto-generated email. Please do not reply.
#         </p>
#       </body>
#     </html>
#     """
#     email_subject = "🔴 Databricks Cost Alert"
# else:
#     # Simple Notification Body
#     email_body = f"""
#     <html>
#       <body style="font-family: Arial, sans-serif; font-size: 15px; color: #333;">
#         <p>Dear Team,</p>

#         <p style="color: green; font-weight: bold;">
#           ✅ Today's cost is within expected range across all workspaces.
#         </p>

#         <p style="margin-top: 20px;">
#           For more details, <a href="{dashboard_link}" style="color: #004080; font-weight: bold;">click here</a>.
#         </p>

#         <p style="margin-top: 20px;">
#           Regards,<br>
#           <strong>Team Exponentia</strong>
#         </p>

#         <p style="color: #888; font-size: 13px; border-top: 1px solid #ccc; padding-top: 10px;">
#           This is an auto-generated email. Please do not reply.
#         </p>
#       </body>
#     </html>
#     """
#     email_subject = "✅ Databricks Cost Check - All OK"



#     send_email(
#         to_email=['shripati.more@exponentia.ai'],
#         subject='Databricks Cost Alert',
#         body=email_body
#     )

# COMMAND ----------

# ----------------------------------------
# 🔁 Imports
# ----------------------------------------
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

# ----------------------------------------
# 📧 Email Function with Error Logging
# ----------------------------------------
def send_email(to_email, subject, body):
    smtp_server = 'smtp.office365.com'
    smtp_port = 587
    sender_email = 'syed.sardar@exponentia.ai'
    password = 'zcxsjpyttdvjsphl'  # ✅ Make sure this is an active app password

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = ", ".join(to_email)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'html'))

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.set_debuglevel(1)  # Debug output to notebook
            server.starttls()
            server.login(sender_email, password)
            server.send_message(msg)
        print("✅ Email sent successfully.")
    except Exception as e:
        print(f"❌ Error sending email: {e}")

# ----------------------------------------
# 🧮 Query to Get Cost Data
# ----------------------------------------
query = """
SELECT 
  workspace, 
  SUM(today) AS Todays_cost, 
  SUM(mtd) AS month_till_date_cose, 
  10 AS expectd_cast
FROM main.default.workspace_cost_details_mv
GROUP BY workspace
"""

df = spark.sql(query)
results = df.collect()

# ----------------------------------------
# 📊 Filter rows where today's cost exceeds expected
# ----------------------------------------
rows_to_include = [row for row in results if row['Todays_cost'] > row['expectd_cast']]

# ----------------------------------------
# 📈 Dashboard Link
# ----------------------------------------
dashboard_link = "https://adb-3859797657626704.4.azuredatabricks.net/dashboardsv3/01f04b37da0f11c29b7becaa4cee4759/published?o=3859797657626704&f_page1%7Es1ParamTimeKey=Week&f_page1%7Es1ParamGroupKey=Workspace"

# ----------------------------------------
# 📤 Compose Email Based on Data
# ----------------------------------------

if rows_to_include:
    # Prepare rows in HTML table
    rows_html = ""
    for row in rows_to_include:
        rows_html += f"""
            <tr>
                <td style="border: 1px solid #aaa; text-align: left; padding: 4px;">{row['workspace']}</td>
                <td style="border: 1px solid #aaa; text-align: right; padding: 4px;">{row['Todays_cost']:.2f}</td>
                <td style="border: 1px solid #aaa; text-align: right; padding: 4px;">{row['expectd_cast']:.2f}</td>
                <td style="border: 1px solid #aaa; text-align: right; padding: 4px;">{row['month_till_date_cose']:.2f}</td>
            </tr>
        """

    email_body = f"""
    <html>
      <body style="font-family: Arial, sans-serif; font-size: 15px; color: #333;">
        <p>Dear Team,</p>

        <p>
          Below is the overview of cost usage <strong>workspace-wise</strong>, where the actual spend is higher than expected.
        </p>

        <h3 style="color: #b30000;">🔴 Databricks Cost Alert</h3>

        <table border="1" cellpadding="6" cellspacing="0" style="border-collapse: collapse; width: 95%; font-size: 14px;">
          <thead>
            <tr style="background-color: #f2f2f2;">
              <th style="border: 1px solid #aaa; text-align: left;">Workspace</th>
              <th style="border: 1px solid #aaa; text-align: right;">Today's Cost (₹)</th>
              <th style="border: 1px solid #aaa; text-align: right;">Expected Cost (₹)</th>
              <th style="border: 1px solid #aaa; text-align: right;">Month Till Date Cost (₹)</th>
            </tr>
          </thead>
          <tbody>
            {rows_html}
          </tbody>
        </table>

        <p style="margin-top: 20px;">
          For more details, <a href="{dashboard_link}" style="color: #004080; font-weight: bold;">click here</a>.
        </p>

        <p style="margin-top: 30px;">
          Regards,<br>
          <strong>Team Exponentia</strong>
        </p>

        <p style="color: #888; font-size: 13px; border-top: 1px solid #ccc; padding-top: 10px;">
          This is an auto-generated email. Please do not reply.
        </p>
      </body>
    </html>
    """
    email_subject = "🔴 Databricks Cost Alert"
else:
    # Green status email
    email_body = f"""
    <html>
      <body style="font-family: Arial, sans-serif; font-size: 15px; color: #333;">
        <p>Dear Team,</p>

        <p style="color: green; font-weight: bold;">
          ✅ Today's cost is within expected range across all workspaces.
        </p>

        <p style="margin-top: 20px;">
          For more details, <a href="{dashboard_link}" style="color: #004080; font-weight: bold;">click here</a>.
        </p>

        <p style="margin-top: 20px;">
          Regards,<br>
          <strong>Team Exponentia</strong>
        </p>

        <p style="color: #888; font-size: 13px; border-top: 1px solid #ccc; padding-top: 10px;">
          This is an auto-generated email. Please do not reply.
        </p>
      </body>
    </html>
    """
    email_subject = "✅ Databricks Cost Check - All OK"

# ----------------------------------------
# ✉️ Send Email
# ----------------------------------------
send_email(
    to_email=['shripati.more@exponentia.ai'],
    subject=email_subject,
    body=email_body
)


In [0]:
# ----------------------------------------
# 🔁 Imports
# ----------------------------------------
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

# ----------------------------------------
# 📧 Email Function with Error Logging
# ----------------------------------------
def send_email(to_email, subject, body):
    smtp_server = 'smtp.office365.com'
    smtp_port = 587
    sender_email = 'syed.sardar@exponentia.ai'
    password = 'zcxsjpyttdvjsphl'  # ✅ Make sure this is an active app password

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = ", ".join(to_email)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'html'))

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.set_debuglevel(1)  # Debug output to notebook
            server.starttls()
            server.login(sender_email, password)
            server.send_message(msg)
        print("✅ Email sent successfully.")
    except Exception as e:
        print(f"❌ Error sending email: {e}")

# ----------------------------------------
# 🧮 Query to Get Cost Data
# ----------------------------------------
query = """
SELECT 
  workspace, 
  SUM(today) AS Todays_cost, 
  SUM(mtd) AS month_till_date_cose, 
  10 AS expectd_cast
FROM main.default.workspace_cost_details_mv
GROUP BY workspace
"""

df = spark.sql(query)
results = df.collect()

# ----------------------------------------
# 📊 Filter rows where today's cost exceeds expected
# ----------------------------------------
rows_to_include = [row for row in results if row['Todays_cost'] > row['expectd_cast']]

# ----------------------------------------
# 📈 Dashboard Link
# ----------------------------------------
dashboard_link = "https://adb-3859797657626704.4.azuredatabricks.net/dashboardsv3/01f04b37da0f11c29b7becaa4cee4759/published?o=3859797657626704&f_page1%7Es1ParamTimeKey=Week&f_page1%7Es1ParamGroupKey=Workspace"

# ----------------------------------------
# 📤 Compose Email Based on Data
# ----------------------------------------

if rows_to_include:
    # Prepare rows in HTML table
    rows_html = ""
    for row in rows_to_include:
        rows_html += f"""
            <tr>
                <td style="border: 1px solid #aaa; text-align: left; padding: 4px;">{row['workspace']}</td>
                <td style="border: 1px solid #aaa; text-align: right; padding: 4px;">{row['Todays_cost']:.2f}</td>
                <td style="border: 1px solid #aaa; text-align: right; padding: 4px;">{row['expectd_cast']:.2f}</td>
                <td style="border: 1px solid #aaa; text-align: right; padding: 4px;">{row['month_till_date_cose']:.2f}</td>
            </tr>
        """

    email_body = f"""
    <html>
      <body style="font-family: Arial, sans-serif; font-size: 15px; color: #333;">
        <p>Dear Team,</p>

        <p>
          Below is the overview of cost usage <strong>workspace-wise</strong>, where the actual spend is higher than expected.
        </p>

        <h3 style="color: #b30000;">🔴 Databricks Cost Alert</h3>

        <table border="1" cellpadding="6" cellspacing="0" style="border-collapse: collapse; width: 95%; font-size: 14px;">
          <thead>
            <tr style="background-color: #f2f2f2;">
              <th style="border: 1px solid #aaa; text-align: left;">Workspace</th>
              <th style="border: 1px solid #aaa; text-align: right;">Today's Cost (₹)</th>
              <th style="border: 1px solid #aaa; text-align: right;">Expected Cost (₹)</th>
              <th style="border: 1px solid #aaa; text-align: right;">Month Till Date Cost (₹)</th>
            </tr>
          </thead>
          <tbody>
            {rows_html}
          </tbody>
        </table>

        <p style="margin-top: 20px;">
          For more details, <a href="{dashboard_link}" style="color: #004080; font-weight: bold;">click here</a>.
        </p>

        <p style="margin-top: 30px;">
          Regards,<br>
          <strong>Team Exponentia</strong>
        </p>

        <p style="color: #888; font-size: 13px; border-top: 1px solid #ccc; padding-top: 10px;">
          This is an auto-generated email. Please do not reply.
        </p>
      </body>
    </html>
    """
    email_subject = "🔴 Databricks Cost Alert"
else:
    # Green status email
    email_body = f"""
    <html>
      <body style="font-family: Arial, sans-serif; font-size: 15px; color: #333;">
        <p>Dear Team,</p>

        <p style="color: green; font-weight: bold;">
          ✅ Today's cost is within expected range across all workspaces.
        </p>

        <p style="margin-top: 20px;">
          For more details, <a href="{dashboard_link}" style="color: #004080; font-weight: bold;">click here</a>.
        </p>

        <p style="margin-top: 20px;">
          Regards,<br>
          <strong>Team Exponentia</strong>
        </p>

        <p style="color: #888; font-size: 13px; border-top: 1px solid #ccc; padding-top: 10px;">
          This is an auto-generated email. Please do not reply.
        </p>
      </body>
    </html>
    """
    email_subject = "✅ Databricks Cost Check - All OK"

# ----------------------------------------
# ✉️ Send Email
# ----------------------------------------
send_email(
    to_email=['shripati.more@exponentia.ai'],
    subject=email_subject,
    body=email_body
)
